### Web Crawiling for whiskybase data, with the following objective
#### 1. getting whisky details (object information)
#### 2. getting user details (subject similarity)

In [3]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import re

In [4]:
#Login
username = "nigel_022001@yahoo.com.hk"
password = "Nigel123"

# Set up the webdriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.whiskybase.com/account/login?p=%2F")

# Find the username/email field and send the username to the input field.
uname = driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[1]/div/div[3]/form/div[1]/div/input')
uname.send_keys(username)
pword = driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[1]/div/div[3]/form/div[2]/div/input')
pword.send_keys(password)

# Click sign in button to login the website.
button = driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[1]/div/div[3]/form/div[3]/div/button')
button.click()



====== WebDriver manager ======
Current google-chrome version is 113.0.5672
Get LATEST chromedriver version for 113.0.5672 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/113.0.5672.63/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\thewa\.wdm\drivers\chromedriver\win32\113.0.5672.63]


In [8]:
def crawl_whisky_details(url:str, driver):
    driver.get(url)

    SCROLL_PAUSE_TIME = 1
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom, Wait to load page, Calculate new scroll height and compare with last scroll height
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Extract the basic whisky information and score
    bottle_name = driver.find_element(by = By.XPATH, value='/html/body/div[1]/div[1]/div/div[2]/div[1]/div[1]/div[1]/div[1]/div/header/h1').text
    number_of_rating = driver.find_element(by = By.XPATH, value='/html/body/div[1]/div[1]/div/div[2]/div[1]/div[2]/div[1]/div/div[1]/dl/dd[2]').text
    average_rating = driver.find_element(by = By.XPATH, value='/html/body/div[1]/div[1]/div/div[2]/div[3]/div[1]/div/div[1]/div/p/span/strong').text
    average_price = driver.find_element(by = By.XPATH, value='//*[@id="partial-aggregate-rating"]/dl/dd[1]/span[1]').text
    information = [tasting_note.text for tasting_note in driver.find_elements(By.XPATH, "/html/body/div[1]/div[1]/div/div[2]/div[1]/div[1]/div[2]/div[1]/dl/dd")]
    origin = [tasting_note.text for tasting_note in driver.find_elements(By.XPATH, "//ul[@class='breadcrumb']")][0]
    origin = re.sub(r'(?<![A-Z\W])(?=[A-Z])', ' ', origin[0])

    # Extract user comment and score
    user_names_list = [user_names.text for user_names in driver.find_elements(By.XPATH, "//div[@class='wb--user-list']//th[@scope='row']")]
    tasting_score_list = [tasting_score.text for tasting_score in driver.find_elements(By.XPATH, "//span[@class='wb--note-refinement-rating-value']")] #it is in form of nose, taste, finish
    overall_comment = [tasting_note.text for tasting_note in driver.find_elements(By.XPATH, "//div[@data-translation-field='message']")]
    nose_comment = [nose_note.text for nose_note in driver.find_elements(By.XPATH, "//div[@data-translation-field='nose_text']")]
    taste_comment = [taste_note.text for taste_note in driver.find_elements(By.XPATH, "//div[@data-translation-field='taste_text']")]
    finish_comment = [finish_note.text for finish_note in driver.find_elements(By.XPATH, "//div[@data-translation-field='finish_text']")]

    # Extract TASTINGTAGS
    flavour_list  = [flavour.text for flavour in driver.find_elements(By.XPATH, "//a[@class='btn-tastingtag']")]
    flavour_vote  = [flavour.get_attribute('data-num') for flavour in driver.find_elements(By.XPATH, "//a[@class='btn-tastingtag']")]

    average_rating = driver.find_element(by = By.XPATH, value='/html/body/div[1]/div[1]/div/div[2]/div[3]/div[1]/div/div[1]/div/p/span/strong').text

    whisky_dict = {'name':bottle_name, 'no_of_rating':number_of_rating, 'rating':average_rating, 
                   'price':average_price, 'full_info': information, 'origin': origin, 
                   'user_names_list': user_names_list, 
                   'tasting_score': tasting_score_list,
                   'overall_comment_list': overall_comment,
                   'nose_comment_list': nose_comment, 
                   'taste_comment_list': taste_comment, 
                   'finish_comment_list': finish_comment, 
                   'flavour_dict': dict(zip(flavour_list,flavour_vote)) }

    return whisky_dict

scotish_whisky_region = ['Campbeltown', 'Highlands', 'Islands', 'Islay', 'Lowlands', 'Speyside'] #for changing origin mapping
whisky_test_dict = crawl_whisky_details('https://www.whiskybase.com/whiskies/whisky/41156/aberlour-12-year-old', driver)


In [1]:
def get_user_preference(userid, driver):
    'result is a list of whisky details list'
    #open user page
    user_url = 'https://www.whiskybase.com/profile/' + userid + '/lists'    
    driver.get(user_url)

    #extract bottle details
    user_wine_list = driver.find_elements(By.XPATH, "/html/body/div[1]/div[1]/div/div[4]/table/tbody/tr")
    user_wine_details = []  #Whisky, Stated Age, Strength, Size, Number of bottles, Casknumber, Votes, Rating, My rating

    for wine in user_wine_list :
        dummy_list = []
        wine_details_cols = wine.find_elements(By.TAG_NAME, "td")
        for details in wine_details_cols:
            dummy_list.append(details.text)
        dummy_list.pop(-1)
        dummy_list.pop(0)
        user_wine_details.append(dummy_list)
        
    return user_wine_details


In [ ]:
#attempt to modulize info
class whisky:
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def myfunc(self):
        print("Hello my name is " + self.name)

p1 = Person("John", 36)

print(p1.name)
print(p1.age)

In [107]:
distiller_origin = dict()
distiller_url = list()

whisky_distiller_url = 'https://www.whiskybase.com/whiskies/distilleries'
driver.get(whisky_distiller_url)

whisky_distiller_crawl = driver.find_elements(By.XPATH, '//*[@id="compositor-material"]/table/tbody/tr')
for whisky_distiller in whisky_distiller_crawl:
    distiller_vote = 0
    try: 
        distiller_vote = int(whisky_distiller.find_elements(By.TAG_NAME, "td")[3].text)    
        distiller_name = whisky_distiller.find_elements(By.TAG_NAME, "td")[0].text    
        distiller_country = whisky_distiller.find_elements(By.TAG_NAME, "td")[1].text       
        distiller_origin[distiller_name] = distiller_country
        link = whisky_distiller.find_elements(By.TAG_NAME, "a")[0].get_attribute('href')
        distiller_url.append(link)
    except ValueError:
        pass    

distiller_origin

{"'t Koelschip": 'Netherlands',
 '1000 Hills Distillery': 'Rwanda',
 '10th Mountain': 'United States',
 '10th Street Distillery': 'United States',
 '1769 Distillery Inc.': 'Canada',
 '23rd Street Distillery': 'Australia',
 '2bar Spirits': 'United States',
 '3 Badge Beverage': 'United States',
 '3 Howls': 'United States',
 '45th Parallel': 'United States',
 '66 Gilead Distillery': 'Canada',
 '8 Doors Distillery': 'Scotland',
 'A. Roborel de Climens': 'France',
 'A. Smith Bowman': 'United States',
 'Aber Falls Whisky Distillery': 'United Kingdom',
 'Aberfeldy': 'Scotland',
 'Aberlour': 'Scotland',
 'Abhainn Dearg': 'Scotland',
 'Acha': 'Spain',
 'Ad Gefrin': 'United Kingdom',
 'Adams Distillery': 'Australia',
 'Adelaide Hills': 'Australia',
 'Adnams (Copper House Distillery)': 'United Kingdom',
 'Aecht Schlenkerla': 'Germany',
 'Agárdi Pálinkafőzde': 'Hungary',
 'Agder Brenneri': 'Norway',
 'AGITATOR': 'Sweden',
 'Aichinger': 'Austria',
 'Ailsa Bay': 'Scotland',
 'Aioi Unibio Co. Ltd.': 

In [116]:
distiller = 'https://www.whiskybase.com/whiskies/distillery/48/aberlour'
whisky_bottle_url_test = list()

driver.get(distiller)
    
whisky_bottle_crawl = driver.find_elements(By.XPATH, '//*[@id="compositor-material"]/table/tbody/tr')
    
for whisky_bottle in whisky_bottle_crawl:    
    try:
        bottle_vote = whisky_bottle.find_elements(By.TAG_NAME, "td")[7]
        number = float(bottle_vote.text)   
        link = whisky_bottle.find_elements(By.CLASS_NAME, "clickable")[0].get_attribute('href')
        whisky_bottle_url_test.append(link)
    except :
        pass    

whisky_bottle_url_test


['https://www.whiskybase.com/whiskies/whisky/220244/aberlour-10-year-old',
 'https://www.whiskybase.com/whiskies/whisky/213952/aberlour-12-year-old',
 'https://www.whiskybase.com/whiskies/whisky/44847/aberlour-12-year-old',
 'https://www.whiskybase.com/whiskies/whisky/184493/aberlour-12-year-old',
 'https://www.whiskybase.com/whiskies/whisky/175164/aberlour-12-year-old',
 'https://www.whiskybase.com/whiskies/whisky/230432/aberlour-12-year-old',
 'https://www.whiskybase.com/whiskies/whisky/215730/aberlour-14-year-old',
 'https://www.whiskybase.com/whiskies/whisky/170372/aberlour-14-year-old',
 'https://www.whiskybase.com/whiskies/whisky/178721/aberlour-16-year-old',
 'https://www.whiskybase.com/whiskies/whisky/227193/aberlour-16-year-old',
 'https://www.whiskybase.com/whiskies/whisky/190075/aberlour-16-year-old',
 'https://www.whiskybase.com/whiskies/whisky/46209/aberlour-16-year-old',
 'https://www.whiskybase.com/whiskies/whisky/215645/aberlour-18-year-old',
 'https://www.whiskybase.co

In [111]:
whisky_bottle_url = list()

for distiller in distiller_url:
    print(distiller)
    driver.get(distiller)
    
    whisky_bottle_crawl = driver.find_elements(By.XPATH, '//*[@id="compositor-material"]/table/tbody/tr')
    
    for whisky_bottle in whisky_bottle_crawl:    
        try:
            bottle_vote = whisky_bottle.find_elements(By.TAG_NAME, "td")[7]      
            number = float(bottle_vote.text)   
            link = whisky_bottle.find_elements(By.CLASS_NAME, "clickable")[0].get_attribute('href')
            whisky_bottle_url.append(link)
        except ValueError:
              pass    

whisky_bottle_url

https://www.whiskybase.com/whiskies/distillery/473/t-koelschip
https://www.whiskybase.com/whiskies/distillery/90642/1000-hills-distillery
https://www.whiskybase.com/whiskies/distillery/85454/10th-mountain
https://www.whiskybase.com/whiskies/distillery/90479/10th-street-distillery
https://www.whiskybase.com/whiskies/distillery/89849/1769-distillery-inc
https://www.whiskybase.com/whiskies/distillery/95065/23rd-street-distillery
https://www.whiskybase.com/whiskies/distillery/84873/2bar-spirits
https://www.whiskybase.com/whiskies/distillery/87864/3-badge-beverage
https://www.whiskybase.com/whiskies/distillery/84953/3-howls
https://www.whiskybase.com/whiskies/distillery/85031/45th-parallel
https://www.whiskybase.com/whiskies/distillery/85918/66-gilead-distillery
https://www.whiskybase.com/whiskies/distillery/94658/8-doors-distillery
https://www.whiskybase.com/whiskies/distillery/93509/a-roborel-de-climens
https://www.whiskybase.com/whiskies/distillery/386/a-smith-bowman


IndexError: list index out of range